In [18]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [19]:
webtoons =pd.read_csv('Webtoon Dataset.csv')
print(webtoons.shape)
webtoons.head(1)

(569, 10)


,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her dream of creating incredible videogames. But then l...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/list?title_no=1218


In [20]:
webtoons_df = webtoons[['id','Name', 'Writer', 'Genre', 'Rating',
                 'Subscribers','Likes','Summary']]
webtoons_df.head(1)

,id,Name,Writer,Genre,Rating,Subscribers,Likes,Summary
0,0,Let's Play,Leeanne M. Krecic (Mongie),Romance,9.62,4.2M,30.6M,"She's young, single and about to achieve her dream of creating incredible videogames. But then l..."


In [21]:
pd.set_option('max_colwidth', 100)
webtoons_df[['Genre','Summary']][:1]

,Genre,Summary
0,Romance,"She's young, single and about to achieve her dream of creating incredible videogames. But then l..."


In [22]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(webtoons_df['Genre'])
print(genre_mat.shape)

(569, 22)


In [23]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat,genre_mat)
print(genre_mat.shape)
print(genre_sim[:1])

(569, 22)
[[1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.
  1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.

In [24]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[  0 505  89 332 202  85 329 205 471 472 474 475 476 478 479 480 481 482
  484 216 489 492 493 494 495 498 499 500 503  90 452 451 348 372 141 169
  172 177 178 181 127 126 124 115 191 112 334 111 344 106 104 196 340 337
   97  96 336  94 448 504 477 282 257  42  40 269  19 245  10  35  34 271
   32 239  31  30  29  28 274  15  26  16 278  46   8   1 235   3  49   2
  163 168 167 166 165 256 164 175 162 160 192 193 194 159 158 255 195 161
  190 174 189 176 260 259 173 179 180 261 171 170 258 182 183 184 185 186
  187 188 254 227 197 253 242 241 240 215 238 217 218 219 220 221 237 236
  234 222 223 233 232 231 224 225 230 229 228 156 243 244 203 198 199 252
  251 250 226 200 249 201 248 204 214 247 206 207 208 209 210 211 212 213
  246 157 125 155  79  51  52  53  54  55  56  57  58  59  60  61  62  63
   64  65  66  67  68  69  70  71  72  73  74  75  76  77  50  48  47  21
    4   5   6   7   9  11  12  13  14  17  18  20  22  45  23  24  25  27
   33  36  37  38  39  41  43  44  78 

유사도 높은 ~번 레코드 순서로

In [25]:
def find_sim_webtoon(df, sorted_ind, title_name, top_n=10):

    # 인자로 입력된 webtoons_df DataFrame에서 'Name' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_webtoon = df[df['Name'] == title_name]

    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_webtoon.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [26]:
similar_webtoons = find_sim_webtoon(webtoons_df, genre_sim_sorted_ind, 'True Beauty',10)
similar_webtoons[['Name', 'Rating']]

[[  0 505  89 332 202  85 329 205 471 472]]


,Name,Rating
0,Let's Play,9.62
505,The Kiss Bet,9.69
89,Behind the Curtain,8.41
332,Romance 101,9.81
202,Act Like You Love Me!,9.78
85,Her Bucket List,9.72
329,Your Smile is a Trap,9.79
205,There Must Be Happy Endings,9.83
471,Phase,9.55
472,Blades of Furry,9.61


In [27]:
webtoons_df[['Name','Rating']].sort_values('Rating',ascending=False)[:10]

,Name,Rating
195,Eleceed,9.93
93,Your Letter,9.93
130,Spirit Fingers,9.91
24,Purple Hyacinth,9.91
187,Omniscient Reader,9.90
191,See You in My 19th Life,9.89
413,Wind Breaker,9.89
88,Bastard,9.87
7,Teenage Mercenary,9.87
161,The Remarried Empress,9.87


In [28]:
def find_sim_webtoon(df, sorted_ind, title_name, top_n=10):
    title_webtoon = df[df['Name'] == title_name]
    title_index = title_webtoon.index.values

    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 웹툰 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]

    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('Rating', ascending=False)[:top_n]

similar_webtoons = find_sim_webtoon(webtoons_df, genre_sim_sorted_ind, 'True Beauty',10)
similar_webtoons[['Name', 'Writer', 'Rating','Summary']]

,Name,Writer,Rating,Summary
205,There Must Be Happy Endings,Moonsia,9.83,The day Yeonu's contract marriage came to an end was also the day that she witnessed her soon-to...
332,Romance 101,Namsoo,9.81,"Bareum's planner might be full, but one thing she desperately needs in her diary is a date. When..."
329,Your Smile is a Trap,Momozerii,9.79,Ex-idol trainee Kiyu has lived his whole life adored by everyone for his good looks. But he drea...
202,Act Like You Love Me!,XUANN,9.78,"25-year-old Ji-eon is stressed, overworked, and penniless. When a mix-up at a temp job has her s..."
484,Choco Latte,Ann,9.76,"Join high schoolers Kyle and Lilly as they discover hope, joy, and love in the simplicities of e..."
480,Half-Ghost,Sejung,9.74,Can she see them or not? Hayeon has always felt something creepy near her. She knows ghosts exis...
475,Swimming Lessons for a Mermaid,YONGCHAN,9.73,"Choa is a mermaid who can't swim! Discouraged, she trades the ocean for a pair of human legs. Bu..."
474,AntiSTALKER,VOKIVORMOK,9.72,At this school usual students study with vampires from wealthy families. She is a representative...
85,Her Bucket List,Vicente Miranda,9.72,Rari's entire world falls apart when her boyfriend dies. Overwhelmed with the pain of losing the...
482,Shadow Bride,Sae In Lee / Soo Bean Lim,9.70,Yunjo is a young prince who has always served his kingdom from behind the scenes. He's spent his...
